In [1]:
from tkinter import *
from tkinter import filedialog
import cv2
import numpy as np


In [2]:
def JanelaFoto():
    janelaFoto = Tk()
    janelaFoto.title('Tela de foto')

    texto_orientacao = Label(janelaFoto, text ='Selecione uma foto para redimensionar ')
    texto_orientacao.grid (column = 0, row = 0)

    voltar= Button(janelaFoto, text='Voltar', command=janelaFoto.destroy)
    voltar.grid(row = 1, column = 0)

    selecionarfoto=(Button(janelaFoto, text='Selecione uma foto para redimensionar', command=SelecionarFoto))
    selecionarfoto.grid(row=1, column=1)

In [3]:
def janelaVideo():
    janelaVideo = Tk()
    janelaVideo.title('Tela de video')

    texto_orientacao=Label(janelaVideo,text= "Selecione um video para redimensionar")
    texto_orientacao.grid(row = 0, column = 0)

    voltar= Button(janelaVideo, text='Voltar', command=janelaVideo.destroy)
    voltar.grid(row = 1, column = 0)

    video=Button(janelaVideo,text ='Selecionar video',command=selecionarVideo)
    video.grid(row = 1, column = 1)

    execVideo=Button(janelaVideo,text = 'Execute o video selecionado',command=executionVideo)
    execVideo.grid(row = 1, column = 2)


In [4]:
def selecionarVideo():

    global video
    arquivo = filedialog.askopenfilename(initialdir='/home/carlo/Desktop', title='Selecionar video',
                                         filetypes=(('mp4', '*.mp4'),('mp4','*mp4')))
    if arquivo:
        print(f'Arquivo selecionado: {arquivo}')
        video= cv2.VideoCapture(arquivo)
        if not video.isOpened():
            print("Erro ao realizar o video.")
            video=None
    else:
        print("Nenhuma foto selecionado!!!")

In [5]:
def executionVideo():
    global video
    if video is None:
        print("Nenhum video foi selecionado!")
    if not video.isOpened():
        print("Erro ao realizar o video, verifique se está carregado ou o vídeo é inválido")
        return

    k_laplace = np.array([[0,1,0],[1,-4,1],[0,1,0]])
    k_blur = np.array([[1/9,1/9,1/9],[1/9,1/9,1/9],[1/9,1/9,1/9]],dtype=np.float32)
    k_sharpen = np.array([[0,-1,0],[-1,5,-1],[0,-1,0]])
    k_edge_detection = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]])

    modo = 'color'
    n = 0
    frame_inicial = None
    frame_final = None

    x1, y1, x2, y2 = 100, 100, 300, 300
    cap = cv2.VideoCapture(video)
    while True:
        ret, frame = cap.read()
        if not ret:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Isso aqui vai deixar em loop
            continue
            # Isso aqui vão mudar as cores
        if modo == 'gray':
            frame_alterado = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        elif modo == 'binary':
            gray = frame_alterado = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            _, frame_alterado = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY)
        elif modo == 'laplace':
            gray = frame_alterado = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_alterado = cv2.filter2D(gray, -1, k_laplace)
        elif modo == 'blur':
            frame_alterado = cv2.filter2D(frame, -1, k_blur)
        elif modo == 'sharpen':
            frame_alterado = cv2.filter @ D(frame, -1, k_sharpen)
        elif modo == 'edge_detection':
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_alterado = cv2.filter2D(gray, -1, k_edge_detection)
        else:
            frame_alterado = frame

        cv2.imshow('Frame', frame_alterado)

        key = cv2.waitKey(n) & 0xFF

        # Controles do vídeo
        if key == ord('g'):
            modo = 'gray'
        elif key == ord('b'):
            modo = 'binary'
        elif key == ord('c'):
            modo = 'color'
        elif key == ord('l'):
            modo = 'laplace'
        elif key == ord('u'):
            modo = 'blur'
        elif key == ord('h'):
            modo = 'sharpen'
        elif key == ord('j'):
            modo = 'edge_detection'
        elif key == ord('t'):
            n = max(1, n - 1)
        elif key == ord('-'):
            n += 1
        elif key == ord('p'):
            n = 0
        elif key == ord('s'):
            if frame_inicial is not None and frame_final is not None:

                for i in range(frame_inicial, frame_final + 1):
                    cap.set(cv2.CAP_PROP_POS_FRAMES, i)
                    ret, frame_salvar = cap.read()
                    if ret:
                        cv2.imwrite(f'frame_{i}.jpg', frame_salvar[y1:y2, x1:x2])
                print(f"Frames {frame_inicial} a {frame_final} salvos com sucesso!")
            else:

                cv2.imwrite('FrameSalvo.jpg', frame_alterado)
                print("Frame salvo com sucesso!")
        elif key == ord('i'):  # Marca o frame inicial
            frame_inicial = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            print(f"Frame inicial definido como: {frame_inicial}")
        elif key == ord('f'):  # Marca o frame final
            frame_final = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            print(f"Frame final definido como: {frame_final}")
        elif key == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [6]:
def JanelaPrincipal():
    janelaPrincipal = Tk()
    janelaPrincipal.title("Tela de Fotos e videos")

    texto_orientacao = Label(janelaPrincipal, text= "Clique alguns dos 2 botões para ir em alguma janelaPrincipal") #
    texto_orientacao.grid(column=0, row=0)

    botao = Button(janelaPrincipal, text="Clique no botao para ir na janela de foto",command= JanelaFoto)
    botao.grid(column=0, row=1) #           Coluna:
                            #       | m0,0  | m0,1  | m0,2  | Rown= Linha
                            #Linha: | m1,0  | m1,1  | m1,2  | Column= Coluna
                            #       | m2,0  | m2,1  | m2,2  |
    botao2 = Button(janelaPrincipal,text ='Clique no botão para ir na janela de video',command=janelaVideo)
    botao2.grid(column=0, row=3)

    Saida=Button(janelaPrincipal, text="Saida", command=janelaPrincipal.quit)
    Saida.grid(column=0, row=2)

    janelaPrincipal.mainloop()


In [7]:
JanelaPrincipal()

Nenhuma foto selecionado!!!
